# Atividade 1 - Treinar um Modelo de Machine Learning

**Curso:** Qualificação em IA Industrial  
**Unidade Curricular:** Machine Learning  
**Objetivo:** Treinar um modelo XGBoost para predição de atrasos de voos

## Objetivo da Atividade
Conduzir a exploração, preparação e configuração para treinar um modelo XGBoost usando o dataset `flights_delays_120.csv` para prever se um voo irá atrasar.

### Etapas da Atividade:
1. **Importação e Exploração dos Dados**
2. **Preparação e Pré-processamento dos Dados**
3. **Treinamento do Modelo XGBoost**
4. **Avaliação do Modelo**

## 1. Importação das Bibliotecas

Primeiro, vamos importar todas as bibliotecas necessárias para o projeto:

In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


## 2. Carregamento e Exploração dos Dados

Agora vamos carregar o dataset e realizar uma análise exploratória inicial:

In [2]:
# Carregamento do dataset
df = pd.read_csv('flights_delays_120.csv')

print("Dataset carregado com sucesso!")
print(f"Dimensões do dataset: {df.shape}")
print("\nPrimeiras 5 linhas do dataset:")
df.head()

Dataset carregado com sucesso!
Dimensões do dataset: (120, 7)

Primeiras 5 linhas do dataset:


,airline,origin,destination,departure_hour,day_of_week,weather,delayed
0,TravelAir,GIG,FOR,11,5,Storm,0
1,JetCloud,CNF,SSA,11,3,Wind,0
2,SkyWings,POA,SSA,4,5,Fog,1
3,JetCloud,BSB,FOR,6,4,Storm,1
4,JetCloud,GRU,FOR,3,1,Rain,1


In [3]:
# Análise exploratória detalhada
print("=== INFORMAÇÕES GERAIS DO DATASET ===")
print(f"Número de linhas: {df.shape[0]}")
print(f"Número de colunas: {df.shape[1]}")
print(f"Colunas: {list(df.columns)}")

print("\n=== INFORMAÇÕES SOBRE OS TIPOS DE DADOS ===")
print(df.info())

print("\n=== VERIFICAÇÃO DE VALORES AUSENTES ===")
print(df.isnull().sum())

print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
print(df.describe())

=== INFORMAÇÕES GERAIS DO DATASET ===
Número de linhas: 120
Número de colunas: 7
Colunas: ['airline', 'origin', 'destination', 'departure_hour', 'day_of_week', 'weather', 'delayed']

=== INFORMAÇÕES SOBRE OS TIPOS DE DADOS ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   airline         120 non-null    object
 1   origin          120 non-null    object
 2   destination     120 non-null    object
 3   departure_hour  120 non-null    int64 
 4   day_of_week     120 non-null    int64 
 5   weather         120 non-null    object
 6   delayed         120 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 6.7+ KB
None

=== VERIFICAÇÃO DE VALORES AUSENTES ===
airline           0
origin            0
destination       0
departure_hour    0
day_of_week       0
weather           0
delayed           0
dtype: int64

=== ESTATÍSTICAS DESCRI

In [4]:
# Análise da variável alvo (delayed)
print("=== ANÁLISE DA VARIÁVEL ALVO (DELAYED) ===")
print(f"Distribuição da variável 'delayed':")
print(df['delayed'].value_counts())
print(f"\nProporção de atrasos:")
print(df['delayed'].value_counts(normalize=True))

print("\n=== ANÁLISE DAS VARIÁVEIS CATEGÓRICAS ===")
categorical_columns = ['airline', 'origin', 'destination', 'weather']

for col in categorical_columns:
    print(f"\n{col.upper()}:")
    print(f"Valores únicos: {df[col].nunique()}")
    print(f"Valores: {df[col].unique()}")

=== ANÁLISE DA VARIÁVEL ALVO (DELAYED) ===
Distribuição da variável 'delayed':
delayed
0    68
1    52
Name: count, dtype: int64

Proporção de atrasos:
delayed
0    0.566667
1    0.433333
Name: proportion, dtype: float64

=== ANÁLISE DAS VARIÁVEIS CATEGÓRICAS ===

AIRLINE:
Valores únicos: 5
Valores: ['TravelAir' 'JetCloud' 'SkyWings' 'FlyFast' 'AirOne']

ORIGIN:
Valores únicos: 5
Valores: ['GIG' 'CNF' 'POA' 'BSB' 'GRU']

DESTINATION:
Valores únicos: 5
Valores: ['FOR' 'SSA' 'CWB' 'BEL' 'REC']

WEATHER:
Valores únicos: 5
Valores: ['Storm' 'Wind' 'Fog' 'Rain' 'Clear']


## 3. Preparação e Pré-processamento dos Dados

Agora vamos preparar os dados para o treinamento do modelo:

In [5]:
# Separação das features (X) e da variável alvo (y)
print("=== SEPARAÇÃO DE FEATURES E VARIÁVEL ALVO ===")

# Definindo X (features) e y (target)
X = df.drop('delayed', axis=1)
y = df['delayed']

print(f"Shape de X (features): {X.shape}")
print(f"Shape de y (target): {y.shape}")
print(f"Colunas das features: {list(X.columns)}")

print("\nPrimeiras 5 linhas das features:")
print(X.head())

=== SEPARAÇÃO DE FEATURES E VARIÁVEL ALVO ===
Shape de X (features): (120, 6)
Shape de y (target): (120,)
Colunas das features: ['airline', 'origin', 'destination', 'departure_hour', 'day_of_week', 'weather']

Primeiras 5 linhas das features:
     airline origin destination  departure_hour  day_of_week weather
0  TravelAir    GIG         FOR              11            5   Storm
1   JetCloud    CNF         SSA              11            3    Wind
2   SkyWings    POA         SSA               4            5     Fog
3   JetCloud    BSB         FOR               6            4   Storm
4   JetCloud    GRU         FOR               3            1    Rain


In [6]:
# Conversão de variáveis categóricas em numéricas usando get_dummies
print("=== CONVERSÃO DE VARIÁVEIS CATEGÓRICAS ===")

print("Colunas antes da conversão:")
print(f"Tipos de dados: \n{X.dtypes}")

# Aplicando get_dummies para converter variáveis categóricas em numéricas
X_encoded = pd.get_dummies(X, drop_first=True)

print(f"\nShape após conversão: {X_encoded.shape}")
print(f"Novas colunas criadas: {X_encoded.shape[1] - X.shape[1]}")
print(f"\nPrimeiras colunas após conversão:")
print(list(X_encoded.columns))

print("\nPrimeiras 5 linhas após conversão:")
print(X_encoded.head())

=== CONVERSÃO DE VARIÁVEIS CATEGÓRICAS ===
Colunas antes da conversão:
Tipos de dados: 
airline           object
origin            object
destination       object
departure_hour     int64
day_of_week        int64
weather           object
dtype: object

Shape após conversão: (120, 18)
Novas colunas criadas: 12

Primeiras colunas após conversão:
['departure_hour', 'day_of_week', 'airline_FlyFast', 'airline_JetCloud', 'airline_SkyWings', 'airline_TravelAir', 'origin_CNF', 'origin_GIG', 'origin_GRU', 'origin_POA', 'destination_CWB', 'destination_FOR', 'destination_REC', 'destination_SSA', 'weather_Fog', 'weather_Rain', 'weather_Storm', 'weather_Wind']

Primeiras 5 linhas após conversão:
   departure_hour  day_of_week  airline_FlyFast  airline_JetCloud  \
0              11            5            False             False   
1              11            3            False              True   
2               4            5            False             False   
3               6            4  

In [7]:
# Divisão dos dados em conjuntos de treinamento e teste
print("=== DIVISÃO DOS DADOS ===")

# Dividindo os dados em treino e teste (80% treino, 20% teste)
# Usando stratify para manter a proporção das classes
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Tamanho do conjunto de treinamento: {X_train.shape}")
print(f"Tamanho do conjunto de teste: {X_test.shape}")

print(f"\nDistribuição das classes no conjunto de treinamento:")
print(y_train.value_counts(normalize=True))

print(f"\nDistribuição das classes no conjunto de teste:")
print(y_test.value_counts(normalize=True))

=== DIVISÃO DOS DADOS ===
Tamanho do conjunto de treinamento: (96, 18)
Tamanho do conjunto de teste: (24, 18)

Distribuição das classes no conjunto de treinamento:
delayed
0    0.5625
1    0.4375
Name: proportion, dtype: float64

Distribuição das classes no conjunto de teste:
delayed
0    0.583333
1    0.416667
Name: proportion, dtype: float64


## 4. Treinamento do Modelo XGBoost

Agora vamos treinar o modelo XGBoost:

In [8]:
# Instanciação e treinamento do modelo XGBoost
print("=== TREINAMENTO DO MODELO XGBOOST ===")

# Instanciando o classificador XGBoost
model = XGBClassifier(
    random_state=42,
    eval_metric='logloss'  # Métrica de avaliação
)

print("Modelo XGBoost instanciado com sucesso!")
print(f"Parâmetros do modelo: {model.get_params()}")

# Treinando o modelo
print("\nIniciando o treinamento do modelo...")
model.fit(X_train, y_train)
print("Treinamento concluído com sucesso!")

=== TREINAMENTO DO MODELO XGBOOST ===
Modelo XGBoost instanciado com sucesso!
Parâmetros do modelo: {'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'feature_weights': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters

## 5. Avaliação do Modelo

Vamos avaliar a performance do modelo treinado:

In [9]:
# Realizando predições no conjunto de teste
print("=== PREDIÇÕES DO MODELO ===")

# Fazendo predições
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

print(f"Número de predições realizadas: {len(y_pred)}")
print(f"Predições únicas: {np.unique(y_pred)}")
print(f"Probabilidades - Shape: {y_pred_proba.shape}")

# Mostrando algumas predições
print(f"\nPrimeiras 10 predições:")
print(f"Reais:      {list(y_test.head(10))}")
print(f"Predições:  {list(y_pred[:10])}")

=== PREDIÇÕES DO MODELO ===
Número de predições realizadas: 24
Predições únicas: [0 1]
Probabilidades - Shape: (24, 2)

Primeiras 10 predições:
Reais:      [1, 0, 1, 0, 0, 1, 0, 1, 0, 1]
Predições:  [np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(1)]


In [ ]:
# Calculando a acurácia
print("=== ACURÁCIA DO MODELO ===")

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.4f} ({accuracy*100:.2f}%)")

# Acurácia no conjunto de treinamento (para comparação)
y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")

# Verificando se há overfitting
if train_accuracy - accuracy > 0.05:
    print("AVISO: Possível overfitting detectado!")
else:
    print("Modelo com boa generalização!")

=== ACURÁCIA DO MODELO ===
Acurácia do modelo: 0.8750 (87.50%)
Acurácia no conjunto de treinamento: 1.0000 (100.00%)
⚠️  Possível overfitting detectado!


In [11]:
# Relatório de classificação
print("=== RELATÓRIO DE CLASSIFICAÇÃO ===")

report = classification_report(y_test, y_pred)
print(report)

# Interpretação do relatório
print("\n=== INTERPRETAÇÃO DO RELATÓRIO ===")
print("Precision: Proporção de predições positivas que foram corretas")
print("Recall: Proporção de casos positivos reais que foram identificados")
print("F1-score: Média harmônica entre precision e recall")
print("Support: Número de exemplos de cada classe no conjunto de teste")

=== RELATÓRIO DE CLASSIFICAÇÃO ===
              precision    recall  f1-score   support

           0       0.92      0.86      0.89        14
           1       0.82      0.90      0.86        10

    accuracy                           0.88        24
   macro avg       0.87      0.88      0.87        24
weighted avg       0.88      0.88      0.88        24


=== INTERPRETAÇÃO DO RELATÓRIO ===
Precision: Proporção de predições positivas que foram corretas
Recall: Proporção de casos positivos reais que foram identificados
F1-score: Média harmônica entre precision e recall
Support: Número de exemplos de cada classe no conjunto de teste


In [12]:
# Matriz de confusão
print("=== MATRIZ DE CONFUSÃO ===")

cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:")
print(cm)

print(f"\nInterpretação da Matriz de Confusão:")
print(f"Verdadeiros Negativos (TN): {cm[0,0]} - Predições corretas de 'não atrasado'")
print(f"Falsos Positivos (FP): {cm[0,1]} - Predições incorretas de 'atrasado'")
print(f"Falsos Negativos (FN): {cm[1,0]} - Predições incorretas de 'não atrasado'")
print(f"Verdadeiros Positivos (TP): {cm[1,1]} - Predições corretas de 'atrasado'")

# Calculando métricas manualmente
total = cm.sum()
tn, fp, fn, tp = cm.ravel()

print(f"\nMétricas calculadas manualmente:")
print(f"Accuracy: {(tp + tn) / total:.4f}")
print(f"Precision: {tp / (tp + fp):.4f}")
print(f"Recall (Sensitivity): {tp / (tp + fn):.4f}")
print(f"Specificity: {tn / (tn + fp):.4f}")

=== MATRIZ DE CONFUSÃO ===
Matriz de Confusão:
[[12  2]
 [ 1  9]]

Interpretação da Matriz de Confusão:
Verdadeiros Negativos (TN): 12 - Predições corretas de 'não atrasado'
Falsos Positivos (FP): 2 - Predições incorretas de 'atrasado'
Falsos Negativos (FN): 1 - Predições incorretas de 'não atrasado'
Verdadeiros Positivos (TP): 9 - Predições corretas de 'atrasado'

Métricas calculadas manualmente:
Accuracy: 0.8750
Precision: 0.8182
Recall (Sensitivity): 0.9000
Specificity: 0.8571


## 6. Análise dos Resultados e Conclusões

Vamos analisar os resultados obtidos e tirar conclusões sobre o modelo:

In [ ]:
# Resumo dos resultados
print("=== RESUMO DOS RESULTADOS ===")
print(f"Dataset processado: {df.shape[0]} registros, {df.shape[1]} colunas")
print(f"Features após encoding: {X_encoded.shape[1]} colunas")
print(f"Divisão dos dados: {X_train.shape[0]} treino, {X_test.shape[0]} teste")
print(f"Modelo XGBoost treinado com sucesso")
print(f"Acurácia obtida: {accuracy:.4f} ({accuracy*100:.2f}%)")

print(f"\n=== ANÁLISE DA PERFORMANCE ===")
if accuracy >= 0.85:
    print("EXCELENTE: Modelo com alta acurácia!")
elif accuracy >= 0.75:
    print("BOM: Modelo com boa acurácia!")
elif accuracy >= 0.65:
    print("REGULAR: Modelo precisa de melhorias!")
else:
    print("BAIXO: Modelo precisa de revisão significativa!")

print(f"\n=== PRÓXIMOS PASSOS SUGERIDOS ===")
print("1. Ajustar hiperparâmetros do XGBoost")
print("2. Realizar feature engineering")
print("3. Testar outros algoritmos de classificação")
print("4. Aplicar técnicas de balanceamento de classes")
print("5. Validação cruzada para melhor estimativa da performance")

=== RESUMO DOS RESULTADOS ===
✅ Dataset processado: 120 registros, 7 colunas
✅ Features após encoding: 18 colunas
✅ Divisão dos dados: 96 treino, 24 teste
✅ Modelo XGBoost treinado com sucesso
✅ Acurácia obtida: 0.8750 (87.50%)

=== ANÁLISE DA PERFORMANCE ===
🎯 EXCELENTE: Modelo com alta acurácia!

=== PRÓXIMOS PASSOS SUGERIDOS ===
1. Ajustar hiperparâmetros do XGBoost
2. Realizar feature engineering
3. Testar outros algoritmos de classificação
4. Aplicar técnicas de balanceamento de classes
5. Validação cruzada para melhor estimativa da performance
